# Предобработка данных

Здесь мы собираем все данные (исходную таблицу и сгенерированные фичи) вместе и делим на тренировочную, валидационную и тестовую выборки в двух вариантах: со сбалансированными и несбалансированными классами

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from pymorphy2 import MorphAnalyzer
import nltk
from nltk.corpus import stopwords
import numpy as np

import re

In [2]:
data = "../data/wb_school_task_2.csv.gzip"
data = pd.read_csv(data, compression='gzip')
data = data.drop_duplicates(ignore_index=True)

In [3]:
data.isna().sum()

id1      0
id2      0
id3      0
text     0
f1       0
f2       0
f3       0
f4       0
f5       0
f6       0
f7       0
f8       0
label    0
dtype: int64

## Общие текстовые фичи

In [4]:
def extract_features(s: pd.Series):
    text = s['text']
    word_count = len(re.findall(r'[а-яА-Яa-zA-Z]+', text))
    # есть отзывы, состоящие из смайликов, пробелов или нижних подчеркиваний
    if word_count == 0:
            return pd.Series({
            'id1': s['id1'],
            'id2': s['id2'],
            'id3': s['id3'],
            'text': text,
            'label': s['label'],
            'text_len': len(text),
            'words_count': 0,
            'sentence_count': 0,
            'number_percentage': 0,
            'caps_percentage': 0,
            'is_empty': 1
        })

    return pd.Series({
        'id1': s['id1'],
        'id2': s['id2'],
        'id3': s['id3'],
        'text': text,
        'label': s['label'],
        'text_len': len(text),
        'words_count': word_count,
        'sentence_count': len(re.split(r'[.!?]+', text)),
        'number_percentage': len(re.findall(r'\d+', text)) / word_count,
        'caps_percentage': len(re.findall(r'[А-ЯA-Z]+', text)) / word_count,
        'is_empty': 0
    })

text_features = data.apply(extract_features, axis=1)
text_features

,id1,id2,id3,text,label,text_len,words_count,sentence_count,number_percentage,caps_percentage,is_empty
0,7596126584852021591,173777575861152844,18254136755182295358,"Хотела купить на замену старых,по итогу эти у...",0,96,16,2,0.000000,0.062500,0
1,5327406586753254371,14366783259208998381,2324030737335224220,Запах по сравнению с обычным DOVE никакой. Оно...,1,99,15,3,0.000000,0.266667,0
2,2636757786779776109,16985421652518589249,4904562693381133981,"Кабель подошёл, хорошо работает.",0,32,5,2,0.000000,0.200000,0
3,15432976385014516263,2629180387521298831,13541353053200353836,"Восторг! Очень приятный аромат, держится долго...",1,81,11,3,0.000000,0.181818,0
4,11933982800034911890,14531191561111600318,9121890650636487522,Визуально все хорошо. Не пробовали. Купили в п...,0,52,8,3,0.000000,0.375000,0
...,...,...,...,...,...,...,...,...,...,...,...
3117,764513678578182418,15635360211325277203,6836784353719029392,"Стекло не плохое,но есть один минус на техно с...",0,66,13,2,0.076923,0.076923,0
3118,6838893897482150395,13016265854619171030,12090430373311552618,"Всё пришло запаковано, ничего не сломано. Лучш...",0,63,10,2,0.000000,0.200000,0
3119,3080528623596565085,17787644417937804433,18003133089489520237,"Мне нравятся 👍🏻 я очень аллергичная, посоветов...",0,101,13,1,0.000000,0.076923,0
3120,13059704988549832630,997055248102065549,12214993272083833542,"Спасибо, целая пришла. Хорошо упаковано",0,40,5,2,0.000000,0.400000,0


In [5]:
merged_dataset = pd.merge(data, text_features, on=['text', 'label', 'id1', 'id2', 'id3'], how='left')
merged_dataset

,id1,id2,id3,text,f1,f2,f3,f4,f5,f6,f7,f8,label,text_len,words_count,sentence_count,number_percentage,caps_percentage,is_empty
0,7596126584852021591,173777575861152844,18254136755182295358,"Хотела купить на замену старых,по итогу эти у...",1,1,1.000000,11,13,4.272727,18,36,0,96,16,2,0.000000,0.062500,0
1,5327406586753254371,14366783259208998381,2324030737335224220,Запах по сравнению с обычным DOVE никакой. Оно...,10,6,2.700000,46,4,4.434783,90,109,1,99,15,3,0.000000,0.266667,0
2,2636757786779776109,16985421652518589249,4904562693381133981,"Кабель подошёл, хорошо работает.",4,4,4.000000,14,0,4.500000,4,6,0,32,5,2,0.000000,0.200000,0
3,15432976385014516263,2629180387521298831,13541353053200353836,"Восторг! Очень приятный аромат, держится долго...",6,6,5.000000,374,30,4.772727,14,15,1,81,11,3,0.000000,0.181818,0
4,11933982800034911890,14531191561111600318,9121890650636487522,Визуально все хорошо. Не пробовали. Купили в п...,19,18,5.000000,6,4,5.000000,26,33,0,52,8,3,0.000000,0.375000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,764513678578182418,15635360211325277203,6836784353719029392,"Стекло не плохое,но есть один минус на техно с...",2,2,3.000000,19,1,4.263158,9,9,0,66,13,2,0.076923,0.076923,0
3118,6838893897482150395,13016265854619171030,12090430373311552618,"Всё пришло запаковано, ничего не сломано. Лучш...",8,8,4.375000,131,83,4.183206,18,18,0,63,10,2,0.000000,0.200000,0
3119,3080528623596565085,17787644417937804433,18003133089489520237,"Мне нравятся 👍🏻 я очень аллергичная, посоветов...",13,12,5.000000,145,2,4.944828,12,14,0,101,13,1,0.000000,0.076923,0
3120,13059704988549832630,997055248102065549,12214993272083833542,"Спасибо, целая пришла. Хорошо упаковано",3,3,5.000000,205,19,4.648780,103,155,0,40,5,2,0.000000,0.400000,0


In [6]:
merged_dataset.isna().sum()

id1                  0
id2                  0
id3                  0
text                 0
f1                   0
f2                   0
f3                   0
f4                   0
f5                   0
f6                   0
f7                   0
f8                   0
label                0
text_len             0
words_count          0
sentence_count       0
number_percentage    0
caps_percentage      0
is_empty             0
dtype: int64

## Фичи из ID

In [7]:
def id_analize(id_col: str, data: pd.DataFrame):
    df = data[[id_col, "label"]]
    df = df.groupby(by=id_col)
    col0 = id_col + "_0"
    col1 = id_col + "_1"
    result = pd.DataFrame(data = {id_col: data[id_col].unique()}, columns=[id_col, col0, col1])
    for idn, table in df:
        if table.shape[0] < 3: 
            result.loc[result[id_col]==idn, col0] = 0
            result.loc[result[id_col]==idn, col1] = 0
            continue
        result.loc[result[id_col]==idn, col0] = (table['label'] == 0).sum()/table.shape[0]
        result.loc[result[id_col]==idn, col1] = (table['label'] == 1).sum()/table.shape[0]
    return result


for col in ['id1', 'id2']:
    id_analized = id_analize(col, data)
    id_analized.to_csv(f"./{col}_data.csv", index=False)
    merged_dataset = merged_dataset.merge(id_analized, how='left', on=col)

In [8]:
merged_dataset

,id1,id2,id3,text,f1,f2,f3,f4,f5,f6,...,text_len,words_count,sentence_count,number_percentage,caps_percentage,is_empty,id1_0,id1_1,id2_0,id2_1
0,7596126584852021591,173777575861152844,18254136755182295358,"Хотела купить на замену старых,по итогу эти у...",1,1,1.000000,11,13,4.272727,...,96,16,2,0.000000,0.062500,0,0,0,0,0
1,5327406586753254371,14366783259208998381,2324030737335224220,Запах по сравнению с обычным DOVE никакой. Оно...,10,6,2.700000,46,4,4.434783,...,99,15,3,0.000000,0.266667,0,0,0,0,0
2,2636757786779776109,16985421652518589249,4904562693381133981,"Кабель подошёл, хорошо работает.",4,4,4.000000,14,0,4.500000,...,32,5,2,0.000000,0.200000,0,0,0,0,0
3,15432976385014516263,2629180387521298831,13541353053200353836,"Восторг! Очень приятный аромат, держится долго...",6,6,5.000000,374,30,4.772727,...,81,11,3,0.000000,0.181818,0,0,0,0,0
4,11933982800034911890,14531191561111600318,9121890650636487522,Визуально все хорошо. Не пробовали. Купили в п...,19,18,5.000000,6,4,5.000000,...,52,8,3,0.000000,0.375000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,764513678578182418,15635360211325277203,6836784353719029392,"Стекло не плохое,но есть один минус на техно с...",2,2,3.000000,19,1,4.263158,...,66,13,2,0.076923,0.076923,0,0,0,0,0
3118,6838893897482150395,13016265854619171030,12090430373311552618,"Всё пришло запаковано, ничего не сломано. Лучш...",8,8,4.375000,131,83,4.183206,...,63,10,2,0.000000,0.200000,0,0,0,0,0
3119,3080528623596565085,17787644417937804433,18003133089489520237,"Мне нравятся 👍🏻 я очень аллергичная, посоветов...",13,12,5.000000,145,2,4.944828,...,101,13,1,0.000000,0.076923,0,0,0,0,0
3120,13059704988549832630,997055248102065549,12214993272083833542,"Спасибо, целая пришла. Хорошо упаковано",3,3,5.000000,205,19,4.648780,...,40,5,2,0.000000,0.400000,0,0,0,0,0


In [9]:
merged_dataset.isna().sum()

id1                  0
id2                  0
id3                  0
text                 0
f1                   0
f2                   0
f3                   0
f4                   0
f5                   0
f6                   0
f7                   0
f8                   0
label                0
text_len             0
words_count          0
sentence_count       0
number_percentage    0
caps_percentage      0
is_empty             0
id1_0                0
id1_1                0
id2_0                0
id2_1                0
dtype: int64

## Фичи из текста

In [10]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors

from navec import Navec

navec = Navec.load("./navec_hudlit_v1_12B_500K_300d_100q.tar")

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [11]:
# лемматизация предложений
falls = 0

def lemmatize(doc):
    global falls
    doc = re.sub(patterns, ' ', doc)
    vector = np.array([0.]*300)
    for token in doc.split():
        token = re.sub("[^А-Яа-я]", '', token)
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            try:
                vector += navec[token]
            except:
                print(token)
                falls += 1
    return vector

emb = data.apply(lambda x: lemmatize(x['text']), axis=1)

ужаснотупой
кутикула
парфюмировать
подошлый
всама
топорщить
дороговатый
ровеня
стёганка
примерочный
оригенать
оыоырыгыоцрцныоцн
застёгиваться
мкостя
большеватый
оститься
наджный
ужасзаказать
продовца
вообщий
большемерить
свитшота
ситочка
лофер
перезаказать
селтомикс
подошлый
фатиновый
провть
подошлый
оочень
размерный
подошлый
футзалка
кленок
поставщиик
будт
большемерить
дублнка
неогранённый
маломерить
красовка
красовка
йня
маломерить
совешенно
курточный
звзд
большимерить
малюхастенький
лочко
фоторамка
пришол
маломерить
лгкий
сандали
разрещё
трусик
маловатый
тплый
идт
чрный
зелный
скатёрка
ребнок
расчта
вообщий
возврать
перезаказать
идт
подойдт
подошол
маломерить
маломерить
большемерить
подкладочка
внешка
безбрендовый
одекватный
нержавейка
нержавейка
нержавейка
вольтаж
каподастр
маломерить
подстрела
трусик
щикарно
бодить
придт
подошол
пупырка
тплый
превзошлый
кросовка
неоригинальный
дшевый
силик
оригин
сооответствовать
тплый
размерный
маломерить
перезаказать
брачок
тяжлай
гумить
пюрешка

In [12]:
emb = emb.apply(lambda x: pd.Series(x, index=('emb_'+str(i) for i in range(300))))

In [13]:
falls

1291

In [14]:
full_data = merged_dataset.merge(emb, left_index=True, right_index=True)
full_data

,id1,id2,id3,text,f1,f2,f3,f4,f5,f6,...,emb_290,emb_291,emb_292,emb_293,emb_294,emb_295,emb_296,emb_297,emb_298,emb_299
0,7596126584852021591,173777575861152844,18254136755182295358,"Хотела купить на замену старых,по итогу эти у...",1,1,1.000000,11,13,4.272727,...,-1.320689,-2.135379,1.128070,0.306198,-0.553014,-1.418026,-0.796837,-0.541121,-2.051006,1.358313
1,5327406586753254371,14366783259208998381,2324030737335224220,Запах по сравнению с обычным DOVE никакой. Оно...,10,6,2.700000,46,4,4.434783,...,-0.531704,-0.973496,1.978720,1.449632,1.901835,-2.893545,0.641482,-2.258740,0.261281,3.434436
2,2636757786779776109,16985421652518589249,4904562693381133981,"Кабель подошёл, хорошо работает.",4,4,4.000000,14,0,4.500000,...,0.321338,-0.223078,0.230015,0.189034,1.090016,-0.539450,-0.076904,-0.128687,0.027489,-0.021773
3,15432976385014516263,2629180387521298831,13541353053200353836,"Восторг! Очень приятный аромат, держится долго...",6,6,5.000000,374,30,4.772727,...,0.084164,-0.608264,-0.235097,-0.194039,-0.152545,-4.316463,-0.421544,-1.331150,0.672442,4.034151
4,11933982800034911890,14531191561111600318,9121890650636487522,Визуально все хорошо. Не пробовали. Купили в п...,19,18,5.000000,6,4,5.000000,...,-0.243878,-0.892752,0.434705,-0.331708,-1.091178,-1.710893,-0.327450,-0.578020,-1.017150,0.951984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,764513678578182418,15635360211325277203,6836784353719029392,"Стекло не плохое,но есть один минус на техно с...",2,2,3.000000,19,1,4.263158,...,0.940821,0.940217,1.410091,0.193880,0.902663,-0.478618,0.997304,-0.452195,0.141350,0.584897
3118,6838893897482150395,13016265854619171030,12090430373311552618,"Всё пришло запаковано, ничего не сломано. Лучш...",8,8,4.375000,131,83,4.183206,...,-0.329931,-0.242179,1.256510,0.229728,-0.099362,-1.064999,0.652931,-1.402012,-0.546669,1.193478
3119,3080528623596565085,17787644417937804433,18003133089489520237,"Мне нравятся 👍🏻 я очень аллергичная, посоветов...",13,12,5.000000,145,2,4.944828,...,-0.685852,-0.067623,1.561003,0.053190,0.832552,-2.953540,-0.032993,-2.010630,-0.976347,1.707753
3120,13059704988549832630,997055248102065549,12214993272083833542,"Спасибо, целая пришла. Хорошо упаковано",3,3,5.000000,205,19,4.648780,...,-0.915561,-0.350195,0.178936,1.223606,-0.427568,-1.506575,0.591230,-1.656120,-1.044409,0.358196


In [16]:
full_data.isna().sum()

id1        0
id2        0
id3        0
text       0
f1         0
          ..
emb_295    0
emb_296    0
emb_297    0
emb_298    0
emb_299    0
Length: 323, dtype: int64

In [17]:
full_data.to_csv("./unbalanced_data/full_data.csv")

In [18]:
balanced_data = pd.concat([full_data, full_data[full_data['label']==1], full_data[full_data['label']==1]], ignore_index=True) 

In [30]:
balanced_data.isna().sum().sum()

0

In [31]:
balanced_data.to_csv("./balanced_data/full_data.csv", index=False)

# Разделение данных

In [34]:
X_train, X_test = train_test_split(balanced_data, test_size=0.2, stratify=balanced_data['label'])
len(X_train), len(X_test)

(3817, 955)

In [35]:
X_train, X_valid = train_test_split(X_train, test_size=0.2, stratify=X_train['label'])
len(X_train), len(X_valid)

(3053, 764)

In [36]:
X_valid.isna().sum().sum()

0

In [37]:
X_train.to_csv("./balanced_data/train_data.csv", index=False)
X_test.to_csv("./balanced_data/test_data.csv", index=False)
X_valid.to_csv("./balanced_data/valid_data.csv", index=False)

# CatBoost

In [3]:
from catboost import CatBoostClassifier
import optuna
import pandas as pd

In [29]:
data = pd.read_csv('./balanced_data/train_data.csv').dropna()
Y_train = data['label']
X_train = data.drop(['label', 'id1', 'id2', 'id3', 'text'], axis=1)
len(X_train)

/tmp/ipykernel_213172/1952649991.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./balanced_data/train_data.csv').dropna()


3052

In [30]:
data = pd.read_csv('./balanced_data/valid_data.csv').dropna()
Y_valid = data['label']
X_valid = data.drop(['label', 'id1', 'id2', 'id3', 'text'], axis=1)
len(X_valid)

764

In [16]:
Y_valid.isna().sum()

0

In [56]:
def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 50, 100),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 10, 15),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        eval_metric='F1',
        verbose=False,
        random_state = 42
    )
    model.fit(X_train, Y_train,
              eval_set=(X_valid, Y_valid),
              verbose=False)
    return model.score(X_valid, Y_valid)

In [57]:
study = optuna.create_study(direction='maximize', study_name='classifier_CatBoost')
study.optimize(objective, n_trials=25)

[I 2023-06-01 20:12:16,710] A new study created in memory with name: classifier_CatBoost
[I 2023-06-01 20:13:36,370] Trial 0 finished with value: 0.8952879581151832 and parameters: {'iterations': 71, 'learning_rate': 0.004545721620752276, 'depth': 11, 'l2_leaf_reg': 1.3836603752836118e-06, 'random_strength': 5.748757736306584e-06}. Best is trial 0 with value: 0.8952879581151832.
[I 2023-06-01 20:16:32,208] Trial 1 finished with value: 0.8965968586387435 and parameters: {'iterations': 77, 'learning_rate': 0.09341091566937756, 'depth': 12, 'l2_leaf_reg': 0.002502932635531173, 'random_strength': 2.1812855596123583e-05}. Best is trial 1 with value: 0.8965968586387435.
[I 2023-06-01 20:31:21,722] Trial 2 finished with value: 0.8900523560209425 and parameters: {'iterations': 83, 'learning_rate': 0.0015392936284404648, 'depth': 14, 'l2_leaf_reg': 0.0002712603080262026, 'random_strength': 0.06655074698318464}. Best is trial 1 with value: 0.8965968586387435.
[I 2023-06-01 20:33:31,517] Trial 3 

KeyboardInterrupt: 

In [58]:
study.best_params

{'iterations': 88,
 'learning_rate': 0.012860393808500103,
 'depth': 14,
 'l2_leaf_reg': 2.076486762606605e-05,
 'random_strength': 0.7091732927390172}

In [59]:
model = CatBoostClassifier(**{'iterations': 88,
                              'learning_rate': 0.012860393808500103,
                              'depth': 14,
                              'l2_leaf_reg': 2.076486762606605e-05,
                              'random_strength': 0.7091732927390172,
                              'random_state': 21, 
                              'eval_metric':'F1'})

In [60]:
model.fit(X_train, Y_train, eval_set=(X_valid, Y_valid))

0:	learn: 0.9227884	test: 0.8258575	best: 0.8258575 (0)	total: 11.2s	remaining: 16m 11s
1:	learn: 0.9706717	test: 0.8407186	best: 0.8407186 (1)	total: 21.2s	remaining: 15m 10s
2:	learn: 0.9832013	test: 0.8348837	best: 0.8407186 (1)	total: 31.9s	remaining: 15m 4s
3:	learn: 0.9939816	test: 0.8461538	best: 0.8461538 (3)	total: 43s	remaining: 15m 3s
4:	learn: 0.9946151	test: 0.8457944	best: 0.8461538 (3)	total: 53.8s	remaining: 14m 53s
5:	learn: 0.9962049	test: 0.8528389	best: 0.8528389 (5)	total: 1m 4s	remaining: 14m 46s
6:	learn: 0.9968414	test: 0.8581315	best: 0.8581315 (6)	total: 1m 15s	remaining: 14m 37s
7:	learn: 0.9977897	test: 0.8674419	best: 0.8674419 (7)	total: 1m 26s	remaining: 14m 26s
8:	learn: 0.9974732	test: 0.8701754	best: 0.8701754 (8)	total: 1m 37s	remaining: 14m 16s
9:	learn: 0.9993691	test: 0.8701754	best: 0.8701754 (8)	total: 1m 48s	remaining: 14m 6s
10:	learn: 0.9993691	test: 0.8705882	best: 0.8705882 (10)	total: 1m 59s	remaining: 13m 57s
11:	learn: 0.9993691	test: 0.8

In [61]:
X_test = pd.read_csv('./unbalanced_data/test_data.csv')
Y_test = X_test['label']
X_test = X_test.drop(['label', 'id1', 'id2', 'id3', 'text'], axis=1)

In [62]:
model.score(X_test, Y_test)

0.968

In [64]:
model.predict(X_test.loc[:10]), Y_test.loc[:10].to_numpy()

(array([1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1.]),
 array([1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]))

In [69]:
X_test = pd.read_csv('./balanced_data/test_data.csv')
Y_test = X_test['label']
X_test = X_test.drop(['label', 'id1', 'id2', 'id3', 'text'], axis=1)

In [70]:
model.score(X_test, Y_test)

0.9099476439790576

In [71]:
model.predict(X_test.loc[:10]), Y_test.loc[:10].to_numpy()

(array([0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.]),
 array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1]))

In [68]:
model.save_model("../model/catboost.model")